**背景:**

承接demo_v1，完善上一篇被简化的部分。逐步代码模块化。

In [24]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

print("version of tensorflow:", tf.__version__)

version of tensorflow: 2.4.1


# 预备

## mask & padding 

可以参考[官方guide](https://www.tensorflow.org/guide/keras/understanding_masking_and_padding#passing_mask_tensors_directly_to_layers).

对不等长的序列需要padding，一般是末尾用0补齐。

In [25]:
raw_inputs = [
    [711, 632, 71],
    [73, 8, 3215, 55, 927],
    [83, 91, 1, 645, 1253, 927],
]

# tensorflow 2.12使用下面的代码
# padded_inputs = tf.keras.utils.pad_sequences(
#     raw_inputs, padding="post"
# )
padded_inputs = tf.keras.preprocessing.sequence.pad_sequences(
    raw_inputs, padding="post"
)
print(padded_inputs)

[[ 711  632   71    0    0    0]
 [  73    8 3215   55  927    0]
 [  83   91    1  645 1253  927]]


对序列做embedding，设置mask_zero=True，查看mask的效果。  

In [26]:
embedding = layers.Embedding(input_dim=5000, output_dim=16, mask_zero=True)
masked_output = embedding(padded_inputs)

print(masked_output._keras_mask)

tf.Tensor(
[[ True  True  True False False False]
 [ True  True  True  True  True False]
 [ True  True  True  True  True  True]], shape=(3, 6), dtype=bool)


**备注：**  
1. 在使用tf.keras的Sequence或者Functional API时，mask layer或者指明了mask的embedding layer的下游，只要支持mask都会自动使用这个信息。
2. 可以认为embedding是mask的生产者，它实现了compute_mask方法供调用；rnn或者lstm等是mask的消费者，他们的__call__方法里支持mask参数，可以手动传进去。 

  
**回到DIN的实现上，sequence特征涉及embedding之后的pooling操作，以及attention部分，都需要处理mask的问题。**

## 定义feature column参数

### dense feature 

### sparse feature

### 

## 定义模型结构参数